In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.svm import SVR
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'

#### 장애인 이용 10월 데이터

In [ ]:
oct=pd.read_csv('oct.csv',thousands=',')
aug=pd.read_csv('aug.csv',thousands=',')
sept=pd.read_csv('sept.csv',thousands=',')

In [ ]:
def modify(oct):
    oct['정류장'].loc[oct['정류장']=='서울']='서울역'
    oct['정류장'].loc[oct['정류장']=='동대문역사문화공원(DDP)']='동대문역사문화공원'
    oct['정류장'].loc[oct['정류장']=='낙성대(강감찬)']='낙성대'
    oct['정류장'].loc[oct['정류장']=='하남시청(덕풍신장)']='하남시청(덕풍-신장)'



modify(oct)
modify(aug)
modify(sept)

#### 지하철 노선 데이터+환승정보 (순서대로 돌려야됨)

In [ ]:
subway=pd.read_csv("서울시 역사마스터 정보.csv",encoding='cp949')
subway

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
5,4926,사우(김포시청),김포골드라인,37.620249,126.719731
6,4925,걸포북변,김포골드라인,37.631650,126.705975
7,4924,운양,김포골드라인,37.653867,126.683930
8,4923,장기,김포골드라인,37.643986,126.669017
9,4922,마산,김포골드라인,37.640732,126.644344


In [ ]:
duplicates = subway[subway.duplicated('역사명', keep=False)]
duplicate_counts = duplicates.groupby('역사명').size().reset_index(name='호선_개수')
subway=pd.merge(subway,duplicate_counts,on='역사명',how='outer')
subway=subway.fillna(1)

In [ ]:
subway["노선앞글자"] = subway["호선"].apply(lambda x:x[0])
subway['노선'] = pd.to_numeric(subway['노선앞글자'], errors='coerce')
subway=subway.dropna()
subway['노선']=subway['노선'].astype(int)
subway['호선_개수']=subway['호선_개수'].astype(int)
subway

,역사_ID,역사명,호선,위도,경도,호선_개수,노선앞글자,노선
0,9996,미사,5호선,37.560927,127.193877,2,5,5
1,2563,미사,5호선,37.560927,127.193877,2,5,5
2,9995,강일,5호선,37.557490,127.175930,2,5,5
3,2562,강일,5호선,37.557490,127.175930,2,5,5
6,4102,김포공항,9호선,37.561916,126.802152,5,9,9
7,2513,김포공항,5호선,37.562384,126.801292,5,5,5
33,246,신설동,2호선,37.574747,127.024932,3,2,2
34,156,신설동,1호선,37.576048,127.024634,3,1,1
36,2639,보문,6호선,37.585274,127.019351,2,6,6
38,418,성신여대입구(돈암),4호선,37.592612,127.016441,2,4,4


In [ ]:
#pd.set_option('display.max_rows', None)
#print(oct['정류장'])

In [ ]:
#pd.set_option('display.max_rows', None)
#print(subway['역사명'])

0                    미사
1                    강일
108              중앙보훈병원
109                둔촌오륜
110         올림픽공원(한국체대)
111                한성백제
112                송파나루
113                  석촌
114                석촌고분
115                  삼전
116               종합운동장
117                 봉은사
118                삼성중앙
119                 선정릉
120                  언주
121                 신논현
122                  사평
123               고속터미널
124                 신반포
125                 구반포
126             동작(현충원)
127           흑석(중앙대입구)
128                  노들
129                 노량진
130                  샛강
131                 여의도
132               국회의사당
133                  당산
134                 선유도
135                 신목동
136                  염창
137                  등촌
138                  증미
139                  가양
140                양천향교
141                마곡나루
142                 신방화
143                공항시장
144                김포공항
145                  개화
146            석남(거북시장)
147             

In [ ]:
def merge_sub(oct):
    oct_sub=pd.merge(oct,subway,left_on=['정류장','노선'],right_on=['역사명','노선'])
    oct_sub=oct_sub.drop(['역사_ID','역사명','호선','노선앞글자'],axis=1)
    return(oct_sub)

oct_sub=merge_sub(oct)
sept_sub=merge_sub(sept)
aug_sub=merge_sub(aug)
print(aug_sub.head(5))
print(oct_sub.head(5))

   월   정류장 이용자유형     합계  04   05   06    07    08    09  ...   22   23  00  \
0  8   서울역   장애인  37334  72  486  624  1315  1830  1661  ...  822  349  38   
1  8    시청   장애인  15470   1   44  172   200   251   325  ...  326  110   3   
2  8    종각   장애인  22771   7  203  109   209   307   453  ...  558  227   6   
3  8  종로3가   장애인  36034  20  308  246   315   554   947  ...  901  333  34   
4  8  종로5가   장애인  29199   1  132  168   178   502   921  ...  374  151   3   

   01  노선  02  03         위도          경도  호선_개수  
0   0   1 NaN NaN  37.556228  126.972135      6  
1   0   1 NaN NaN  37.565715  126.977088      2  
2   0   1 NaN NaN  37.570161  126.982923      1  
3   0   1 NaN NaN  37.570406  126.991847      3  
4   0   1 NaN NaN  37.570926  127.001849      1  

[5 rows x 32 columns]
    월   정류장 이용자유형     합계  04   05   06    07    08    09  ...   22   23  00  \
0  10   서울역   장애인  41234  68  490  664  1315  1927  1907  ...  949  386  32   
1  10    시청   장애인  17886   5   47  189   193   204

In [ ]:
oct_sub.to_csv("oct_sub.csv",index = False)
sept_sub.to_csv("sept_sub.csv",index = False)
aug_sub.to_csv("aug_sub.csv",index = False)